## MTA Service Announcement Preprocessing

- MTA Service Alerts: Beginning 2020 can be dound here: https://data.ny.gov/Transportation/MTA-Service-Alerts-Beginning-April-2020/7kct-peq7/about_data 
- Data Dictionary can be downloaded here: https://data.ny.gov/api/views/7kct-peq7/files/b328bd7e-d79f-4344-b6f5-a8dec1897169?download=true&filename=MTA_ServiceAlerts_DataDictionary.pdf
- Data overview can be downloaded here: https://data.ny.gov/api/views/7kct-peq7/files/802a019e-34fb-4bd5-a297-81aa7a183e6f?download=true&filename=MTA_ServiceAlerts_Overview.pdf


In [19]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import to_date, hour
from pyspark.sql.types import DateType, IntegerType
import pandas as pd
import datetime
from urllib.request import urlretrieve

In [20]:
spark = SparkSession.builder \
    .appName("mta_preprocessing") \
    .getOrCreate()

In [21]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema explicitly
schema = StructType([
    StructField("Alert ID", IntegerType(), True),
    StructField("Event ID", IntegerType(), True),
    StructField("Update Number", IntegerType(), True),
    StructField("Date", StringType(), True),
    StructField("Agency", StringType(), True),
    StructField("Status Label", StringType(), True),
    StructField("Affected", StringType(), True),
])

# Read the CSV file with the defined schema
df = spark.read.schema(schema).option("header", "true").csv("../data/landing/mta.csv")
df.show()

+--------+--------+-------------+--------------------+-----------+--------------------+--------------------+
|Alert ID|Event ID|Update Number|                Date|     Agency|        Status Label|            Affected|
+--------+--------+-------------+--------------------+-----------+--------------------+--------------------+
|  337474|  163057|            0|07/30/2024 11:56:...|NYCT Subway|              delays|                   D|
|  337473|  163056|            0|07/30/2024 11:46:...|NYCT Subway|              delays|                   6|
|  337469|  163055|            0|07/30/2024 11:42:...|   NYCT Bus|      part-suspended|                M14D|
|  337471|  163055|            1|07/30/2024 11:44:...|   NYCT Bus|      part-suspended|                M14D|
|  337468|  163054|            0|07/30/2024 11:42:...|   NYCT Bus|              detour|                M14A|
|  337470|  163054|            1|07/30/2024 11:43:...|   NYCT Bus|              detour|                M14A|
|  337467|  163053|

In [22]:
# the file link url was working when working on my project, 
# however, it is now corrupt so data has been downloaded and uploaded directly to the path '../data/landing/mta.csv'
# feel free to uncomment and run the following code in this block to view

# URL of the CSV file
#url = 'https://data.ny.gov/api/views/7kct-peq7/rows.csv?accessType=DOWNLOAD'

# output directory
#output_dir = "../data/landing/mta_service_alert.csv"

# download
#urlretrieve(url, output_dir) 

#dftest = spark.read.schema(schema).option("header", "true").csv("../data/landing/mta_service_alert.csv")
#dftest.show()


In [23]:
df.printSchema()

root
 |-- Alert ID: integer (nullable = true)
 |-- Event ID: integer (nullable = true)
 |-- Update Number: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Status Label: string (nullable = true)
 |-- Affected: string (nullable = true)



In [24]:
df.show()

+--------+--------+-------------+--------------------+-----------+--------------------+--------------------+
|Alert ID|Event ID|Update Number|                Date|     Agency|        Status Label|            Affected|
+--------+--------+-------------+--------------------+-----------+--------------------+--------------------+
|  337474|  163057|            0|07/30/2024 11:56:...|NYCT Subway|              delays|                   D|
|  337473|  163056|            0|07/30/2024 11:46:...|NYCT Subway|              delays|                   6|
|  337469|  163055|            0|07/30/2024 11:42:...|   NYCT Bus|      part-suspended|                M14D|
|  337471|  163055|            1|07/30/2024 11:44:...|   NYCT Bus|      part-suspended|                M14D|
|  337468|  163054|            0|07/30/2024 11:42:...|   NYCT Bus|              detour|                M14A|
|  337470|  163054|            1|07/30/2024 11:43:...|   NYCT Bus|              detour|                M14A|
|  337467|  163053|

In [25]:
df.count()

334431

In [26]:
# Filter based on the minimum possible values for numerical features
df = df.where((F.col('Update Number') >= 0) &
              (F.col('Alert ID') >= 0) &
              (F.col('Event ID') >= 0))

df.count()

334431

In [27]:
# remove unecessary columns
df_converted = df.drop('Event ID').drop('Affected').drop('Status Label')
df_converted.show()
df_converted.printSchema()

+--------+-------------+--------------------+-----------+
|Alert ID|Update Number|                Date|     Agency|
+--------+-------------+--------------------+-----------+
|  337474|            0|07/30/2024 11:56:...|NYCT Subway|
|  337473|            0|07/30/2024 11:46:...|NYCT Subway|
|  337469|            0|07/30/2024 11:42:...|   NYCT Bus|
|  337471|            1|07/30/2024 11:44:...|   NYCT Bus|
|  337468|            0|07/30/2024 11:42:...|   NYCT Bus|
|  337470|            1|07/30/2024 11:43:...|   NYCT Bus|
|  337467|            0|07/30/2024 11:32:...|       LIRR|
|  337464|            0|07/30/2024 11:24:...|       LIRR|
|  337466|            1|07/30/2024 11:29:...|       LIRR|
|  337463|            0|07/30/2024 11:18:...|NYCT Subway|
|  337461|            0|07/30/2024 11:07:...|NYCT Subway|
|  337462|            1|07/30/2024 11:09:...|NYCT Subway|
|  337460|            0|07/30/2024 10:58:...|NYCT Subway|
|  337465|            1|07/30/2024 11:25:...|NYCT Subway|
|  337458|    

In [28]:
# Define the date format used in the CSV
date_format = "MM/dd/yyyy hh:mm:ss a"

# Convert the Date column to timestamp
df_converted = df_converted.withColumn("Date", to_timestamp(col("Date"), date_format))

# Now want to make sure to include necessary dates
# Define the date range
start_date = '2023-07-01'
end_date = '2023-12-31'

# Filter the DataFrame
df_converted = df_converted.filter((col("Date") >= start_date) & (col("Date") <= end_date)).orderBy("Date")

df_converted.show(truncate=False)
df_converted.printSchema()

+--------+-------------+-------------------+-----------+
|Alert ID|Update Number|Date               |Agency     |
+--------+-------------+-------------------+-----------+
|233465  |1            |2023-07-01 00:01:00|NYCT Bus   |
|233466  |2            |2023-07-01 00:05:00|LIRR       |
|233467  |0            |2023-07-01 00:10:00|LIRR       |
|233468  |1            |2023-07-01 00:12:00|NYCT Subway|
|233469  |0            |2023-07-01 00:17:00|NYCT Subway|
|233470  |0            |2023-07-01 00:21:00|LIRR       |
|233471  |0            |2023-07-01 00:23:00|LIRR       |
|233472  |3            |2023-07-01 00:26:00|LIRR       |
|233473  |1            |2023-07-01 00:29:00|NYCT Subway|
|233474  |0            |2023-07-01 00:32:00|NYCT Subway|
|233475  |4            |2023-07-01 00:32:00|LIRR       |
|233476  |5            |2023-07-01 00:40:00|LIRR       |
|233477  |0            |2023-07-01 00:55:00|NYCT Subway|
|233478  |6            |2023-07-01 01:00:00|LIRR       |
|233479  |1            |2023-07

In [29]:
# remove any rows which involve service announcements affecting the Long Island Rail Road (LIRR) and Metro-North Rail Road (MNR) service as it is not contained in our defined geographical area
df_converted = df_converted.filter(~df_converted["Agency"].contains("LIRR"))
df_converted = df_converted.filter(~df_converted["Agency"].contains("MNR"))

# Extract the date from the 'Date' column
df_converted = df_converted.withColumn('date_only', to_date(df_converted['Date']))

# Extract the hour from the 'Date' column
df_converted = df_converted.withColumn('hour', hour(df_converted['Date']))

df_converted.show(truncate=False)

+--------+-------------+-------------------+-----------+----------+----+
|Alert ID|Update Number|Date               |Agency     |date_only |hour|
+--------+-------------+-------------------+-----------+----------+----+
|233465  |1            |2023-07-01 00:01:00|NYCT Bus   |2023-07-01|0   |
|233468  |1            |2023-07-01 00:12:00|NYCT Subway|2023-07-01|0   |
|233469  |0            |2023-07-01 00:17:00|NYCT Subway|2023-07-01|0   |
|233473  |1            |2023-07-01 00:29:00|NYCT Subway|2023-07-01|0   |
|233474  |0            |2023-07-01 00:32:00|NYCT Subway|2023-07-01|0   |
|233477  |0            |2023-07-01 00:55:00|NYCT Subway|2023-07-01|0   |
|233479  |1            |2023-07-01 01:02:00|NYCT Subway|2023-07-01|1   |
|233480  |0            |2023-07-01 01:03:00|NYCT Subway|2023-07-01|1   |
|233482  |1            |2023-07-01 01:10:00|NYCT Subway|2023-07-01|1   |
|233483  |1            |2023-07-01 01:11:00|NYCT Subway|2023-07-01|1   |
|233487  |2            |2023-07-01 01:25:00|NYCT Su

In [30]:
# Group by 'date_only', 'hour', and 'Agency' and count the occurrences
grouped_df = df_converted.groupBy("date_only", "hour", "Agency").count()

# Pivot the 'Agency' column to create separate columns for each agency
pivot_df = grouped_df.groupBy("date_only", "hour") \
    .pivot("Agency", ["NYCT Bus", "NYCT Subway"]) \
    .agg(F.first("count"))

# Rename the columns to 'bus_count', 'subway_count', and 'bt_count'
pivot_df = pivot_df.withColumnRenamed("NYCT Bus", "bus_count") \
                   .withColumnRenamed("NYCT Subway", "subway_count")

# Fill null values with 0, in case some combinations don't have alerts and therefore have no counts
pivot_df = pivot_df.fillna(0)

# rename dat_only to date for consistency
pivot_df = pivot_df.withColumnRenamed("date_only", "date")

# Show the results
pivot_df.show()

+----------+----+---------+------------+
|      date|hour|bus_count|subway_count|
+----------+----+---------+------------+
|2023-12-28|   9|        0|           7|
|2023-12-30|  23|        1|           5|
|2023-10-10|  10|        8|           5|
|2023-08-04|   0|        5|           6|
|2023-07-25|   9|        4|           2|
|2023-11-01|   8|        1|           3|
|2023-09-16|   8|        1|           0|
|2023-10-21|   9|        2|           0|
|2023-07-08|   4|        0|           9|
|2023-07-05|  18|        2|           6|
|2023-07-09|   2|        0|           4|
|2023-08-21|   7|        0|           6|
|2023-09-29|   5|        3|           2|
|2023-09-21|  12|        5|          12|
|2023-07-14|  11|        9|          12|
|2023-12-07|  17|        2|          13|
|2023-09-19|  14|        1|           4|
|2023-07-18|   8|        1|           5|
|2023-10-05|  17|        0|           5|
|2023-11-17|  15|        0|           8|
+----------+----+---------+------------+
only showing top

In [31]:
# calculating how many date/hour combinations there are in the specified time frame
(30+30+31+31+31+31)*24

4416

In [32]:
# comparing this to our processed dataset
pivot_df.count()

4281

In [33]:
# not all dates are included, now need to include them all
all_dates = pd.date_range(start=start_date, end=end_date).date

# generate all hours
all_hours = list(range(0, 24))

# create DataFrame with all date and hour combinations
all_combinations = [(date, hour) for date in all_dates for hour in all_hours]
all_combinations_df = spark.createDataFrame(all_combinations, ["date", "hour"])

# ensure 'date' is of DateType and 'hour' is of IntegerType
all_combinations_df = all_combinations_df.withColumn("date", col("date").cast(DateType()))
all_combinations_df = all_combinations_df.withColumn("hour", col("hour").cast(IntegerType()))

In [34]:
# perform a full outer join with the original DataFrame
full_df = all_combinations_df.join(pivot_df, on=["date", "hour"], how="left")

# fill missing counts with 0 as there are assumed to be no service alerts at this date/hour combination
full_df = full_df.fillna({"bus_count": 0, "subway_count": 0})

# sort the DataFrame by 'date' and 'hour'
full_df = full_df.orderBy(["date", "hour"])

# show the resulting DataFrame
full_df.show()

+----------+----+---------+------------+
|      date|hour|bus_count|subway_count|
+----------+----+---------+------------+
|2023-07-01|   0|        1|           5|
|2023-07-01|   1|        1|           7|
|2023-07-01|   2|        2|           2|
|2023-07-01|   3|        0|           1|
|2023-07-01|   4|        0|          11|
|2023-07-01|   5|        0|           7|
|2023-07-01|   6|        0|           3|
|2023-07-01|   7|        0|           6|
|2023-07-01|   8|        4|           4|
|2023-07-01|   9|        1|           3|
|2023-07-01|  10|        0|           6|
|2023-07-01|  11|        0|           5|
|2023-07-01|  12|        3|           4|
|2023-07-01|  13|        0|           3|
|2023-07-01|  14|        0|           4|
|2023-07-01|  15|        2|           6|
|2023-07-01|  16|        0|           1|
|2023-07-01|  17|        0|           2|
|2023-07-01|  18|        1|           2|
|2023-07-01|  19|        1|           4|
+----------+----+---------+------------+
only showing top

In [35]:
# checking to see if all dates and hours are now included
# get the number of columns in the DataFrame
num_rows = full_df.count()

# print the number of rows
print(f"Number of rows: {num_rows}")

Number of rows: 4416


In [36]:
# can now save csv to '../data/curated/' directory
full_df.write.mode("overwrite").option("header", "true").csv("../data/curated/hourly_service_counts.csv")